In [ ]:
import os
import pdf2image as p2i
import numpy as np
import pandas as pd
# import pyzbar.pyzbar as zbar
from PIL import Image
import pickle

In [41]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pdf2image as p2i

#### Write code

In [20]:
%%writefile extractor.py

import os
import shutil
import pdf2image as p2i

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def clean(folder):
    """rm -rf folder"""
    shutil.rmtree(folder)

def extract(path, box=(1377,2047,1503,2173), get_key=False, output_folder='default', filename='default', pages=None):
    """Write png files and a csv file to output_folder.
    
    Input:
        path: path of pdf file
        output_folder: name of output folder, default name is same as pdf file
        filename: suffix of output filename, default name is {output_folder}_{}.pgm
        box: (left, top, right, bottom) for cropping checkcode
            108 fake: ???
            108~109 real: (1377,2047,1503,2173)
            a4: (1417.78, 2102.78, 1575.26, 2260.26) not tested
    Note: 
        a4 page size = 29.7 x 21 cm; 11.69 x 8.27 in; 2339 x 1654 in 200 dpi
        1cm = 0.3937 inches = 78.74 pixels in 200 dpi
    Output:
        None
    """
    
    
    # 建立資料夾，預設為 pdf 檔名
    if output_folder == 'default':
        fn = os.path.split(path)[-1]
        output_folder = os.path.splitext(fn)[0] 
    try:
        os.mkdir(output_folder)
    except FileExistsError:
        print("Error: There exists a folder called \'{}\'.".format(output_folder))
        question = "Do you want to remove the folder? [y/N]"
        ans = input(prompt=question)
        if ans in ['y', 'Y']:
            clean(output_folder)
            os.mkdir(output_folder)
        elif ans in ['n', 'N', '', None]:
            return
    
    # 定義每一張檔名，預設為"資料夾名稱_{}.jpg"
    if filename == 'default':
        filename = output_folder + '_{:03d}.png'
    else:
        filename = filename + '_{:03d}.png'
    output_path = os.path.join(output_folder, filename)
    
    # 將 pdf 逐頁拆分並轉換成 jpg 並儲存
    if pages != None:
        imgs = p2i.convert_from_path(path, grayscale = True, 
                                     first_page=pages[0], last_page=pages[1])
    else:
        imgs = p2i.convert_from_path(path, grayscale = True)
    
    ### create csv
    num_imgs = len(imgs)
    df = pd.DataFrame([
            [filename.format(i) for i in range(num_imgs)], 
            [None]*num_imgs
        ])
    df.T.to_csv(os.path.join(output_folder, output_folder+'.csv'), 
                header=False, 
                index=False)
    
    for i, im in enumerate(imgs):
        ### extract checkcode
        checkcode = im.crop(box)
        checkcode = checkcode.resize((28,28))
        checkcode.save(output_path.format(i))
        
        ### extract key
        ### TBD
        
class raw_data:
    def __init__(self, path):
        """initiate the object with path = folder_name"""
        self.path = path
        self.name = os.path.split(path)[-1]
        self.df = pd.read_csv(os.path.join(path, self.name+'.csv'), 
                              header=None)
        self.num = self.df.shape[0]
    
    def examine(self, start=None, end=None, each_row=5, size=2, label='data'):
        if start == None:
            start = 0
        if end == None:
            end = self.num
        if isinstance(label, str) and label == 'data':
            label = self.df

        total = end - start
        rows = total // each_row
        if total % each_row != 0:
            rows += 1
            
        fig,axs = plt.subplots(rows, each_row, 
                               figsize=(size*each_row,size*rows),
                               squeeze=False)

        for k in range(total):
            real_k = start + k
            i,j = k//each_row, k%each_row
            fn = label.iloc[real_k,0]
            ax = axs[i][j]
            ax.axis('off')
            img = plt.imread(os.path.join(self.path, fn))
            ax.imshow(img, cmap='Greys_r', vmin=0, vmax=1)
            ax.set_title('%s'%label.iloc[real_k,1])
            ax.text(0, 27, fn)
        plt.show()
        
        return fig
    
    def labeler(self, start=None, end=None, each_row=5, size=2):
        """API for examining and modifying the data and labels"""
        if start == None:
            start = 0
        if end == None:
            end = self.num

        total = end - start
        rows = total // each_row
        if total % each_row != 0:
            rows += 1
            
        changes = []
        new_df = self.df.copy()
        new_df['notes'] = ''
        i = 0
        
        while True:
            if i >= rows:
                print('Reaching the end.  [s]ave or [q]uit?')
            else:
                self.examine(start + i*each_row, 
                             min(start + (i+1)*each_row, total), 
                             each_row, size, 
                             label=new_df)

                print('Give me five digits for your changes: [h] help')

            c = input()
            if c == 'h':
                print("""h: this page
s: save and leave
q: quit without saving
...1.: change the fourth label to 1
.-...: drop the second data 
(the two lines above can be merged as .-.1.)
empty: default is no change
*** No changes are made until saved ***
""")
                continue
            elif c == 'q':
                break
            elif c == 's':
                self.df = new_df.loc[~(new_df.notes == 'd'),[0,1]]
                for new_i in range(self.num):
                    if new_df.loc[new_i,'notes'] == 'd':
                        fn = new_df.iloc[new_i,0]
                        os.remove(os.path.join(self.path, fn))
                old_num = self.num
                self.num = self.df.shape[0]
                print("Changed %s labels and dropped %s pictures:"
                      %(np.sum(new_df.notes == 'r'),
                        np.sum(new_df.notes == 'd')))
                print("Number of images: %s -> %s"%(old_num, self.num))
                self.df.to_csv(os.path.join(self.path, self.name+'.csv'), 
                               header=False, 
                               index=False)
                print("New %s.csv written to %s."
                      %(self.name, self.path))
                break
            elif c == '': ### default action
                i += 1
                continue
            elif len(c) == each_row:
                changes.append(c)
                for j,d in enumerate(c):
                    real_k = start + i*each_row + j
                    if d == '.':
                        continue
                    elif d == '-':
                        new_df.iloc[real_k,1] = -1
                        new_df.loc[real_k,'notes'] = 'd'
                        continue
                    elif d.isdigit():
                        if new_df.iloc[real_k,1] != float(d):
                            new_df.loc[real_k,'notes'] = 'r' 
                            new_df.iloc[real_k,1] = float(d)
                        continue
                    else:
                        print("Something is wrong.  Try again.")
                        break
            else:
                print("Not valid input.  Press h for help.")
                continue
    
    def merge_to(self, target='nsysu_digits'):
        #shuffle and keep label only
        #TBD
        pass

Overwriting extractor.py


#### Code from Chan-Yu

In [ ]:
### from Chan-Yu

def extractor(path, output_folder = 'default', filename = 'default'):
    
    """
    Input:
        path: path of pdf file
        output_folder: name of output folder, default name is same as pdf file
        filename: name of jpg file, default name is {output_folder}_{}.jpg
    Output:
        None
    """
    
    # 建立資料夾，預設為 pdf 檔名
    if output_folder == 'default':
        tmp = path.split('/')
        tmp = tmp[-1].split('.')
        output_folder = tmp[0]
    try:
        os.mkdir(output_folder)
    except:
        print("Error: There exists a folder called \'{}\'.".format(output_folder))
        return
    
    # 定義每一張檔名，預設為"資料夾名稱_{}.jpg"
    if filename == 'default':
        filename = output_folder + '_{}.jpg'
    else:
        filename = filename + '_{}.jpg'
    output_path = output_folder + '/' + filename
    
    # 將 pdf 逐頁拆分並轉換成 jpg 並儲存
    imgs = p2i.convert_from_path(path, fmt = 'jpg', grayscale = True)
    for i, im in enumerate(imgs):
        im.save(output_path.format(i+1))
    
    n_img = len(imgs)
    checkcode = np.zeros([n_img,120,120])
    qrcode = np.zeros([n_img,400,400])
    for i, im in enumerate(imgs):
        
        # 擷取 checkcode 並記錄至 `checkcode`
        img = np.asarray(im)
        rect = img[img.shape[0]-400:,img.shape[1]-400:]    ### 選取最右下角的 400x400
        rect = rect.mean(axis = -1)
        edge_check = np.where(rect < 100)
        r = edge_check[0].max()
        c = edge_check[1].max()
        rect = rect[r-130:r-10, c-130:c-10]    ### -10 是從邊界往內縮，從右下角選取 120x120（checkcode 大小）
        checkcode[i,:,:] = rect
        
        # 擷取 QR code 並記錄至 `qrcode`
        qr = img[img.shape[0]-400:, :400]    ### 選取最左下角的 400x400
        qr = qr.mean(axis = -1)
        qrcode[i,:,:] = sharpen_vec(qr) ### 把黑色變得更黑
    
    # 將 `checkcode` 儲存，檔名預設為 checkcode_{output_folder}.pkl
    cc_name = 'checkcode_' + output_folder + '.pkl'
    with open(cc_name,'wb') as handle:
        pickle.dump(checkcode, handle, protocol = pickle.HIGHEST_PROTOCOL)
    
    # 將 `qrcode` 儲存，檔名預設為 qrcode{output_folder}.pkl
    qc_name = 'qrcode_' + output_folder + '.pkl'
    with open(qc_name,'wb') as handle:
        pickle.dump(qrcode, handle, protocol = pickle.HIGHEST_PROTOCOL)

    return